In [1]:
import ast
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import warnings

from itertools import product
from joblib import Parallel, delayed

from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler
from stepmix.bootstrap import blrt_sweep
from stepmix.stepmix import StepMix
from tqdm import tqdm

from src.model_fit import do_StepMix
from src.model_select import elbow_method, lrt, blrt_sweep_custom

In [2]:
CVI = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn']
max_threads = 8

# Preparation
## Data

In [3]:
var_list = [
    'clseusa', 'ambornin', 'amcit', 'amlived', 'amenglsh', 'amchrstn',
    'amgovt', 'amfeel', 'amcitizn', 'amshamed', 'belikeus', 'ambetter',
    'ifwrong', 'proudsss', 'proudgrp', 'proudpol', 'prouddem', 'proudeco',
    'proudspt', 'proudart', 'proudhis', 'proudmil', 'proudsci']

var_list_f = [var + "_f" for var in var_list]
var_list_n = [var + "_n" for var in var_list]

ctrl_list = [
    'party_f', 'race_f', 'educ_f', 'region_f', 'reltrad_f', 'religstr_f', 
    'born_usa_f', 'sex_f', 'age_n', 'lnrealinc2004_n', 'age_n', 'lnrealinc2004_n']

In [5]:
# data2004 = pd.read_parquet(f"data/data2004_replic.parquet")
data2004 = pd.read_parquet(f"data/data2004_830.parquet")
data2004_ni = pd.read_parquet(f"data/data2004_830_ni.parquet")

# Datasets with numeric outcomes
data_n = data2004[var_list_n]
data_n = StandardScaler().fit_transform(data_n)
data_n = pd.DataFrame(data_n)

data_ni_n = data2004[var_list_n]
data_ni_n = StandardScaler().fit_transform(data_ni_n)
data_ni_n = pd.DataFrame(data_ni_n)

# Dataset with categorical outcomes and reindexing to 0 (as expected by StepMix)
data_f = data2004[var_list_n] - 1
data_ni_f = data2004_ni[var_list_n] - 1

# Dataset with controls (same as the authors)
controls = data2004[ctrl_list]
controls_dum = pd.get_dummies(controls)

# Sample weights
weights = data2004['wgt']

# Replication

The authors perform LCA with 1 to 8 classes with missing data, sociodemographic covariates, and sample weights. Do we get similar results with the [StepMix](https://stepmix.readthedocs.io/en/latest/api.html#stepmix) package?

In [6]:
results = Parallel(n_jobs=max_threads)(
    delayed(do_StepMix)(
        data = data_ni_f,
        controls = controls_dum,
        n = n_clust,
        msrt = 'categorical_nan',
        covar = 'with',
        weights = weights)
    for n_clust in tqdm(range(1,9), desc='Fitting latent models'))

replic_LCA = pd.DataFrame(results).drop(columns = ['model', 'params', 'silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn'])
replic_LCA['l2_red'] = 100 * (replic_LCA['LL'].iloc[0] - replic_LCA['LL']) / replic_LCA['LL'].iloc[0]

Fitting latent models: 100%|██████████| 8/8 [00:00<00:00, 171.67it/s]


In [7]:
replic_LCA.style

,n_clust,min_clust_size,max_clust_size,aic,bic,sabic,relative_entropy,classif_error,df,LL,l2_red
0,1,830,830,38337.224411,38686.609913,38948.997530,nan,0.000000,74.000000,-23.005557,-0.000000
1,2,278,552,36239.901860,37108.644189,37761.067453,0.858332,0.040978,184.000000,-21.609579,6.068001
2,3,142,366,35505.252173,36893.351329,37935.810241,0.863578,0.059555,294.000000,-21.034489,8.567789
3,4,129,314,35112.055618,37019.511602,38452.006160,0.878059,0.065965,404.000000,-20.665094,10.173469
4,5,60,291,34919.905023,37346.717833,39169.248039,0.900723,0.061068,514.000000,-20.416810,11.252701
5,6,57,261,34959.153889,37905.323526,40117.889379,0.883546,0.080268,624.000000,-20.307924,11.726005
6,7,31,188,34932.508430,38398.034895,41000.636395,0.872202,0.095341,734.000000,-20.159342,12.371856
7,8,25,217,34916.984954,38901.868246,41894.505393,0.890117,0.086774,844.000000,-20.017461,12.988584


## Absolute values

In [8]:
best_aic = replic_LCA.sort_values('aic', ascending=True).iloc[0]
best_bic = replic_LCA.sort_values('bic', ascending=True).iloc[0]
best_sabic = replic_LCA.sort_values('sabic', ascending=True).iloc[0]
best_entropy = replic_LCA.sort_values('relative_entropy', ascending=False).iloc[0]

print(f"Best model according to the min/max rule applied to...")
print(f"- AIC (min) has {best_aic['n_clust']:.0f} clusters")
print(f"- BIC (min) has {best_bic['n_clust']:.0f} clusters")
print(f"- SABIC (min) has {best_sabic['n_clust']:.0f} clusters")
print(f"- Relative entropy (max) has {best_entropy['n_clust']:.0f} clusters")

Best model according to the min/max rule applied to...
- AIC (min) has 8 clusters
- BIC (min) has 3 clusters
- SABIC (min) has 2 clusters
- Relative entropy (max) has 5 clusters


## Elbow method

In [9]:
best_aic = elbow_method(replic_LCA, 'aic')
best_bic = elbow_method(replic_LCA, 'bic')
best_sabic = elbow_method(replic_LCA, 'sabic')
best_entropy = elbow_method(replic_LCA, 'relative_entropy')

print(f"Best model according to the Elbow method applied to...")

if best_aic is None: print("- AIC is None")
else: print(f"- AIC has {best_aic} clusters")

if best_bic is None: print("- BIC is None")
else: print(f"- BIC has {best_bic} clusters")

if best_sabic is None: print("- SABIC is None")
else: print(f"- SABIC has {best_sabic} clusters")
    
if best_entropy is None: print("- Entropy is None")
else: print(f"- Entropy has {best_entropy} clusters")

Best model according to the Elbow method applied to...
- AIC has 3 clusters
- BIC has 2 clusters
- SABIC has 2 clusters
- Entropy has 2 clusters


## LRT
*LRT - not advisable for comparing models with $k$ and $k-1$ classes as the resulting test statistics does not follow the $\chi^2$ distribution under the null hypothesis. The implementation below compare models to the 1-class model, which is sometimes recommended, without a formal justification.*

In [10]:
LRT = lrt(replic_LCA)
LRT = LRT.iloc[1:].reset_index(drop=True)
LRT.style.hide(axis=0)

n clust,L2 reduction,LR ratio,LR pval
2,0.060680,2.791955,1.000000
3,0.085678,3.942135,1.000000
4,0.101735,4.680926,1.000000
5,0.112527,5.177493,1.000000
6,0.117260,5.395266,1.000000
7,0.123719,5.692429,1.000000
8,0.129886,5.976192,1.000000


## BLRT
### Without missing values, covariates and sample weights
Default StepMix implementation

In [16]:
opt_params = {
    'method': 'gradient',
    'intercept': True,
    'max_iter': 500}

latent_mod = StepMix(
    measurement = 'categorical',
    n_init = 7,
    abs_tol = 1e-4,
    rel_tol = 1e-4,
    init_params = 'random',
    structural_params = opt_params,
    progress_bar = 0)

In [ ]:
s_time = time.time()
BLRT = blrt_sweep(
    latent_mod,
    data_f,
    low = 1,
    high = 8,
    n_repetitions = 500)
e_time = time.time()

Testing 1 vs. 2 classes...
Bootstrapping null model...

Bootstrapping estimator...


Bootstrap Repetitions    : 100%|██████████| 500/500 [00:12<00:00, 39.93it/s, max_LL=-1.88e+4, median_LL=-1.9e+4, min_LL=-1.93e+4] 



Bootstrapping alternative model...

Bootstrapping estimator...


Bootstrap Repetitions    : 100%|██████████| 500/500 [00:19<00:00, 25.04it/s, max_LL=-1.87e+4, median_LL=-1.9e+4, min_LL=-1.93e+4]


Testing 2 vs. 3 classes...
Bootstrapping null model...

Bootstrapping estimator...


...

Bootstrap Repetitions    : 100%|██████████| 500/500 [2:01:43<00:00, 14.61s/it, max_LL=-1.67e+4, median_LL=-1.72e+4, min_LL=-1.76e+4]     



Bootstrapping alternative model...

Bootstrapping estimator...


Bootstrap Repetitions    : 100%|██████████| 500/500 [6:59:21<00:00, 50.32s/it, max_LL=-1.67e+4, median_LL=-1.71e+4, min_LL=-1.76e+4]     


Testing 5 vs. 6 classes...
Bootstrapping null model...

Bootstrapping estimator...


Bootstrap Repetitions    :  57%|█████▋    | 285/500 [1:39:51<00:25,  8.52it/s, max_LL=-1.65e+4, median_LL=-1.69e+4, min_LL=-1.74e+4]    

In [13]:
print(f"Total execution time: {(e_time - s_time) / 60:.2f} minutes")

Total execution time: 12.70 minutes


In [14]:
BLRT_res = pd.concat([pd.DataFrame({'p': [np.nan]}), BLRT]).reset_index(drop=True) # Add a row for the saturated model
BLRT_res["n clust"] = [f"{i+1} vs. {i} clust" for i in BLRT_res.index]
BLRT_res = BLRT_res.iloc[1:]
BLRT_res = BLRT_res[["n clust", "p"]]
BLRT_res.to_csv("output/models/BLRT_simplex.csv", index=False)

BLRT_res.style.hide(axis=0).format({"p": "{:.3f}"})

n clust,p
2 vs. 1 clust,0.000
3 vs. 2 clust,0.000
4 vs. 3 clust,0.000
5 vs. 4 clust,0.002
6 vs. 5 clust,0.042
7 vs. 6 clust,0.012
8 vs. 7 clust,0.140


In [15]:
if BLRT_res[BLRT_res['p'] > 0.05].empty:
    best_LCA = None
else:
    best_LCA = BLRT_res[BLRT_res['p'] > 0.05]
    best_LCA = best_LCA.index[0]

print(f"Optimal number of clusters for LCA without covariates and sample weights is {best_LCA} according to BLRT.")

Optimal number of clusters for LCA without covariates and sample weights is 7 according to BLRT.


### With covariates and sample weights
Custom implementation as the wrapper function provided by StepMix does not allow for sample weights. It is much slower and yields opposite results!

In [ ]:
opt_params = {
    'method': 'gradient',
    'intercept': True,
    'max_iter': 500,}

latent_mod = StepMix(
    measurement = 'categorical_nan',
    n_init = 7,
    abs_tol = 1e-4,
    rel_tol = 1e-4,
    init_params = 'random',
    structural = 'covariate',
    structural_params = opt_params,
    progress_bar = 0)

In [ ]:
s_time = time.time()
BLRT = blrt_sweep_custom(
    latent_mod,
    data_ni_f,
    controls_dum,
    sample_weight = weights,
    low = 1,
    high = 8,
    n_repetitions = 500,
    n_jobs = max_threads)
e_time = time.time()

In [ ]:
print(f"Total execution time: {(e_time - s_time) / 60:.2f} minutes")

In [ ]:
BLRT_res = pd.concat([pd.DataFrame({'p': [np.nan]}), BLRT]).reset_index(drop=True) # Add a row for the saturated model
BLRT_res["n clust"] = [f"{i+1} vs. {i} clust" for i in BLRT_res.index]
BLRT_res = BLRT_res.iloc[1:]
BLRT_res = BLRT_res[["n clust", "p"]]
BLRT_res.to_csv("output/models/BLRT_complex.csv", index=False)

BLRT_res.style.hide(axis=0).format({"p": "{:.3f}"})

In [ ]:
if BLRT_res[BLRT_res['p'] > 0.05].empty:
    best_LCA = None
else:
    best_LCA = BLRT_res[BLRT_res['p'] > 0.05]
    best_LCA = best_LCA.index[0]

print(f"Optimal number of clusters for LCA with covariates and sample weights is {best_LCA} according to BLRT.")

# Comparaison

Compare models...
- Categorical (= LCA) or continuous (= LPA / GMM)
- With missing or imputed values
- With or without survey weights
- With or without covariates

Regarding LPA...
- `gaussian_diag_nan` does not converge and `gaussian_diag` tends to overfit. Hence the use of `gaussian_spherical`, which is less flexible. 
- Convergence becomes problematic for LPA above 13-14 classes, making scaling is necessary.

In [ ]:
measurements = ['categorical', 'continuous']
missing_values = ['without', 'with']
covariates = ['without', 'with']
sample_weights = ['without', 'with']

max_clust = 16
clust_range = range(1, max_clust+1)
latent_params = list(product(measurements, missing_values, covariates, sample_weights))
latent_grid = product(clust_range, latent_params)

def set_data(msrt, NAs):
    if msrt == 'categorical' and NAs == 'with':
        return data_ni_f
    elif msrt == 'categorical' and NAs == 'without':
        return data_f
    elif msrt == 'continuous' and NAs == 'with':
        return data_ni_n
    elif msrt == 'continuous' and NAs == 'without':
        return data_n

def set_measurement(msrt, NAs):
    if msrt == 'categorical' and NAs == 'with':
        return 'categorical_nan'
    elif msrt == 'categorical' and NAs == 'without':
        return 'categorical'
    elif msrt == 'continuous' and NAs == 'with':
        return 'gaussian_spherical_nan'
    elif msrt == 'continuous' and NAs == 'without':
        return 'gaussian_spherical'

In [ ]:
results = Parallel(n_jobs = max_threads)(
    delayed(do_StepMix)(
        data = set_data(msrt, NAs),
        controls = controls_dum if covar == 'with' else None,
        n = n_clust,
        msrt = set_measurement(msrt, NAs), 
        covar = covar,
        weights = weights if wgt == 'with' else None)
    for n_clust, (msrt, NAs, covar, wgt) in tqdm(latent_grid, desc = 'Fitting latent models'))

In [ ]:
latent_all = pd.DataFrame(results)

LCA = latent_all[latent_all['params'].apply(lambda x: 'categorical' in x.get('msrt', ''))]
LPA = latent_all[latent_all['params'].apply(lambda x: 'gaussian' in x.get('msrt', ''))]

## LCA

In [ ]:
def extract_label(param_dict):
    return f"msrt:{param_dict.get('msrt')}, NAs:{param_dict.get('NAs')}, covar:{param_dict.get('covar')}, wgt:{param_dict.get('wgt')}"

In [ ]:
row1_metrics = ['sabic', 'aic', 'classif_error', 'LL']
row2_metrics = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn']
all_metrics = row1_metrics + row2_metrics

LCA.loc[:, 'label'] = LCA['params'].apply(extract_label)

# Create a 3-row x 4-column grid
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(22, 18), sharex=True)

axes = axes.flatten()

# Plot metrics in first 8 axes
for i, metric in enumerate(all_metrics):
    ax = axes[i]
    for label, group in LCA.groupby('label'):
        group = group.sort_values('n_clust')
        ax.plot(group['n_clust'], group[metric], marker='', label=label)
    ax.set_title(metric.upper())
    ax.set_xlabel("Number of Clusters")
    ax.set_ylabel(metric.upper())

# Hide any unused subplot (in case)
for j in range(8, 12):
    axes[j].axis('off')

# Legend in the entire third row (axes 8 to 11 merged)
# Remove the 4 axes in row 3 and create one big axis there:
for ax in axes[8:12]:
    fig.delaxes(ax)
legend_ax = fig.add_subplot(3,1,3)  # new axis spanning entire bottom row

legend_ax.axis('off')  # no axis lines or ticks

# Create dummy lines for legend from any group (using first group)
first_label = next(iter(LCA['label'].unique()))
lines = []
labels = []

for label, group in LCA.groupby('label'):
    line, = legend_ax.plot([], [], marker='o', label=label)
    lines.append(line)
    labels.append(label)

legend_ax.legend(lines, labels, loc='center', ncol=4, title="Combination")

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
fig.suptitle("Model Fit and Clustering Metrics vs. Number of Clusters", fontsize=16, y=0.98)

plt.show()

In [ ]:
# Define your metric groups
metrics_row1 = ['aic', 'classif_error', 'LL', 'sabic']
metrics_row2 = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn']
all_metrics = metrics_row1 + metrics_row2

maximize_metrics = {'LL', 'silhouette', 'calinski_harabasz'}
marker_styles = ['D', '*']  # diamond = best, star = 2nd best
marker_alpha = [0.5, 0.8]
marker_size = [90, 80]
LCA.loc[:, 'label'] = LCA['params'].apply(extract_label)
unique_labels = LCA['label'].unique()
colors = {label: plt.cm.tab20(i % 20) for i, label in enumerate(unique_labels)}

# Set up GridSpec with 3 rows: 2 for plots, 1 for legend
fig = plt.figure(figsize=(24, 12))
gs = gridspec.GridSpec(3, 4, height_ratios=[1, 1, 0.1])  # last row is for legend

axes = [fig.add_subplot(gs[i // 4, i % 4]) for i in range(8)]

for i, metric in enumerate(all_metrics):
    ax = axes[i]
    maximize = metric in maximize_metrics

    df_metric = LCA.dropna(subset=[metric])

    sorted_df = (
        df_metric
        .sort_values(metric, ascending=not maximize)
        .groupby("n_clust")
        .head(2)
        .copy()
    )
    sorted_df['rank'] = sorted_df.groupby('n_clust').cumcount()

    for _, row in sorted_df.iterrows():
        marker = marker_styles[row['rank']]
        alpha = marker_alpha[row['rank']]
        size = marker_size[row['rank']]
        ax.scatter(
            row['n_clust'], row[metric],
            color=colors[row['label']],
            marker=marker,
            alpha=alpha,
            s=size,
            label=row['label'] if row['rank'] == 0 else None
        )

    title = "Classification error" if metric == 'classif_error' else metric
    ax.set_title(title.upper())
    # ax.set_xlabel("Number of Clusters")
    # ax.set_ylabel(metric.upper())
    ax.set_ylabel("")

# Create an extra subplot just for the legend
legend_ax = fig.add_subplot(gs[2, :])  # full width on third row
legend_ax.axis('off')

legend_handles = [
    plt.Line2D([0], [0], marker='o', linestyle='None',
               color=colors[label], label=label)
    for label in unique_labels
]
legend_ax.legend(handles=legend_handles, loc='center', ncol=4, title="Params", fontsize='medium')

plt.suptitle("Top 2 Models per Number of Clusters Across Metrics", fontsize=18)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## LPA

In [ ]:
row1_metrics = ['sabic', 'aic', 'classif_error', 'LL']
row2_metrics = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn']
all_metrics = row1_metrics + row2_metrics

LPA.loc[:, 'label'] = LPA['params'].apply(extract_label)

# Create a 3-row x 4-column grid
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(22, 18), sharex=True)

axes = axes.flatten()

# Plot metrics in first 8 axes
for i, metric in enumerate(all_metrics):
    ax = axes[i]
    for label, group in LPA.groupby('label'):
        group = group.sort_values('n_clust')
        ax.plot(group['n_clust'], group[metric], marker='', label=label)
    ax.set_title(metric.upper())
    ax.set_xlabel("Number of Clusters")
    ax.set_ylabel(metric.upper())

# Hide any unused subplot (in case)
for j in range(8, 12):
    axes[j].axis('off')

# Legend in the entire third row (axes 8 to 11 merged)
# Remove the 4 axes in row 3 and create one big axis there:
for ax in axes[8:12]:
    fig.delaxes(ax)
legend_ax = fig.add_subplot(3,1,3)  # new axis spanning entire bottom row

legend_ax.axis('off')  # no axis lines or ticks

# Create dummy lines for legend from any group (using first group)
first_label = next(iter(LPA['label'].unique()))
lines = []
labels = []

for label, group in LPA.groupby('label'):
    line, = legend_ax.plot([], [], marker='o', label=label)
    lines.append(line)
    labels.append(label)

legend_ax.legend(lines, labels, loc='center', ncol=4, title="Combination")

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
fig.suptitle("Model Fit and Clustering Metrics vs. Number of Clusters", fontsize=16, y=0.98)

plt.show()

In [ ]:
# Define your metric groups
metrics_row1 = ['aic', 'classif_error', 'LL', 'sabic']
metrics_row2 = ['silhouette', 'calinski_harabasz', 'davies_bouldin', 'dunn']
all_metrics = metrics_row1 + metrics_row2

maximize_metrics = {'LL', 'silhouette', 'calinski_harabasz'}
marker_styles = ['D', '*']  # diamond = best, star = 2nd best
marker_alpha = [0.5, 0.8]
marker_size = [90, 80]
LPA.loc[:, 'label'] = LPA['params'].apply(extract_label)
unique_labels = LPA['label'].unique()
colors = {label: plt.cm.tab20(i % 20) for i, label in enumerate(unique_labels)}

# Set up GridSpec with 3 rows: 2 for plots, 1 for legend
fig = plt.figure(figsize=(24, 12))
gs = gridspec.GridSpec(3, 4, height_ratios=[1, 1, 0.1])  # last row is for legend

axes = [fig.add_subplot(gs[i // 4, i % 4]) for i in range(8)]

for i, metric in enumerate(all_metrics):
    ax = axes[i]
    maximize = metric in maximize_metrics

    df_metric = LPA.dropna(subset=[metric])

    sorted_df = (
        df_metric
        .sort_values(metric, ascending=not maximize)
        .groupby("n_clust")
        .head(2)
        .copy()
    )
    sorted_df['rank'] = sorted_df.groupby('n_clust').cumcount()

    for _, row in sorted_df.iterrows():
        marker = marker_styles[row['rank']]
        alpha = marker_alpha[row['rank']]
        size = marker_size[row['rank']]
        ax.scatter(
            row['n_clust'], row[metric],
            color=colors[row['label']],
            marker=marker,
            alpha=alpha,
            s=size,
            label=row['label'] if row['rank'] == 0 else None
        )

    title = "Classification error" if metric == 'classif_error' else metric
    ax.set_title(title.upper())
    # ax.set_xlabel("Number of Clusters")
    # ax.set_ylabel(metric.upper())
    ax.set_ylabel("")

# Create an extra subplot just for the legend
legend_ax = fig.add_subplot(gs[2, :])  # full width on third row
legend_ax.axis('off')

legend_handles = [
    plt.Line2D([0], [0], marker='o', linestyle='None',
               color=colors[label], label=label)
    for label in unique_labels
]
legend_ax.legend(handles=legend_handles, loc='center', ncol=4, title="Params", fontsize='medium')

plt.suptitle("Top 2 Models per Number of Clusters Across Metrics", fontsize=18)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()